In [ ]:
#pip install konlpy # 코랩용 모듈 설치문

In [50]:
import pandas as pd
import re
from konlpy.tag import Okt
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [51]:
df = pd.read_csv('data/daum_news_samsung_20230301_20240229.csv')
df_stock = pd.read_csv('data/samsung_stock_20230301_20240228.csv')
df['date'] = pd.to_datetime(df['날짜']).dt.date

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'[^\w\s.,!?%+-]', '', str(text))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_title'] = df['뉴스제목'].apply(clean_text) # 뉴스제목 정제

print(df.head(5))

           날짜                                      뉴스제목        date  \
0  2023-03-01           ‘반도체의 봄’ 언제 오나, 삼성전자 1·2월 적자 3조  2023-03-01   
1  2023-03-01         ‘갤럭시’→‘삼성’ 일본서 8년 만에 제 이름 찾는 삼성전자  2023-03-01   
2  2023-03-01  삼성전자, 중저가 OLED TV 3종 상반기 추가 출시…점유율 확보 속도  2023-03-01   
3  2023-03-01              삼성전자·수도권대기환경청, 미세먼지 저감 협약 체결  2023-03-01   
4  2023-03-01             지난달 국내증시 거래대금 1위는 삼성전자…SM은 4위  2023-03-01   

                               clean_title  
0             반도체의 봄 언제 오나, 삼성전자 12월 적자 3조  
1             갤럭시삼성 일본서 8년 만에 제 이름 찾는 삼성전자  
2  삼성전자, 중저가 OLED TV 3종 상반기 추가 출시점유율 확보 속도  
3              삼성전자수도권대기환경청, 미세먼지 저감 협약 체결  
4             지난달 국내증시 거래대금 1위는 삼성전자SM은 4위  


In [ ]:
okt = Okt() # 우선 Okt 사용
stopwords = {'은', '는', '이', '가', '하', '의', '에', '을', '를', '도', '과', '와', '으로', '로', '에서', '하다', '고'}

def preprocess(text):
    tokens = okt.morphs(text, stem=True)
    meaningful_words = [word for word in tokens if word not in stopwords and len(word) > 1]
    return ' '.join(meaningful_words)

df['processed_title'] = df['clean_title'].apply(preprocess)

print(df['processed_title'])

In [59]:
model_name = "snunlp/KR-FinBERT-SC" # Finbert 모델 지정
tokenizer = AutoTokenizer.from_pretrained(model_name) # 토크나이징
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print(model.config) # 모델 설정 확인
model.to(device) # 모델을 GPU로 이동
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neutral": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20000
}



In [60]:
# 감성 분석 함수
def predict_sentiment(text):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=128, 
        padding=True
    )
    outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1) # 확률값으로 변환
    conf, pred_label = torch.max(probs, dim=1)
    return label_map[pred_label.item()], conf.item()

def get_sentiment_score(text):
    if not text.strip():
        return 0
    label, conf = predict_sentiment(text)

    # 긍부정 확률값 반환
    if label == 'positive':
        return 1 * conf
    elif label == 'negative':
        return -1 * conf
    else:
        return 0


In [ ]:
df['sentiment_score'] = df['clean_title'].progress_apply(get_sentiment_score) # 날짜 기준으로 정렬

daily_sentiment = df.groupby('날짜').agg(
    avg_sentiment=('sentiment_score', 'mean'),
).reset_index()

print(df['sentiment_score'])

0       -0.998721
1        0.000000
2        0.999845
3        0.999656
4        0.976110
5        0.999791
6        0.999412
7        0.901540
8        0.000000
9        0.000000
10       0.000000
11       0.999575
12       0.000000
13       0.000000
14       0.999108
15       0.000000
16       0.999878
17       0.000000
18       0.000000
19       0.000000
20       0.000000
21       0.000000
22      -0.968318
23      -0.912635
24       0.874619
25      -0.935344
26       0.000000
27       0.000000
28       0.000000
29       0.000000
30       0.000000
31      -0.998447
32       0.999851
33       0.000000
34      -0.999735
35      -0.826274
36       0.000000
37       0.000000
38       0.000000
39      -0.899026
40       0.995791
41       0.000000
42       0.000000
43       0.000000
44      -0.932602
45       0.000000
46       0.000000
47       0.000000
48       0.000000
49       0.000000
50       0.999875
51       0.000000
52       0.874970
53       0.000000
54       0.987766
55       0

In [65]:
print(df_stock.isnull().sum())
df_stock.dropna(inplace=True) # 결측치 있는 행 삭제

df_stock = df_stock[df_stock['거래량'] > 0] # 거래량 0인 행 삭제

df_stock = df_stock.sort_values('날짜')

print(df_stock[['날짜', '종가', '거래량', '변동률(%)', '상승 여부']].head(10))

날짜        0
종가        0
거래량       0
변동률(%)    0
상승 여부     0
dtype: int64
           날짜     종가       거래량  변동률(%)  상승 여부
0  2023.03.02  60800  13095682    0.00      0
1  2023.03.03  60500  10711405   -0.49      0
2  2023.03.06  61500  13630602    1.65      1
3  2023.03.07  60700  11473280   -1.30      0
4  2023.03.08  60300  14161857   -0.66      0
5  2023.03.09  60100  14334499   -0.33      0
6  2023.03.10  59500  11902471   -1.00      0
7  2023.03.13  60000  12779724    0.84      1
8  2023.03.14  59000  12147346   -1.67      0
9  2023.03.15  59800  10482149    1.36      1


In [66]:
df_stock = df_stock.loc[:, ~df_stock.columns.duplicated()]

print(df_stock)
print(daily_sentiment)

# 날짜 형식 변호나 & 시간 정규화
daily_sentiment['날짜'] = pd.to_datetime(daily_sentiment['날짜']).dt.normalize()
df_stock['날짜'] = pd.to_datetime(df_stock['날짜']).dt.normalize()

# 날짜 기준으로 두 데이터 내부 조인
df_merged = pd.merge(
    df_stock,
    daily_sentiment,
    on='날짜',
    how='inner',
    suffixes=('_stock', '_sentiment')
)

# 날짜 컬럼 정리 (날짜 겹침)
if '날짜_sentiment' in df_merged.columns:
    df_merged.drop(columns=['날짜_sentiment'], inplace=True)

if '날짜_stock' in df_merged.columns:
    df_merged.rename(columns={'날짜_stock': '날짜'}, inplace=True)

if 'avg_sentiment' in df_merged.columns: # 컬럼명 변경
    df_merged.rename(columns={'avg_sentiment': '감성 점수'}, inplace=True)

df_merged['감성 점수'] = df_merged['감성 점수'].fillna(0) # 결측값 0으로 채우기

df_merged = df_merged.sort_values('날짜')
df_merged['변동률(%)'] = df_merged['변동률(%)'].fillna(0)


             날짜     종가       거래량  변동률(%)  상승 여부
0    2023.03.02  60800  13095682    0.00      0
1    2023.03.03  60500  10711405   -0.49      0
2    2023.03.06  61500  13630602    1.65      1
3    2023.03.07  60700  11473280   -1.30      0
4    2023.03.08  60300  14161857   -0.66      0
5    2023.03.09  60100  14334499   -0.33      0
6    2023.03.10  59500  11902471   -1.00      0
7    2023.03.13  60000  12779724    0.84      1
8    2023.03.14  59000  12147346   -1.67      0
9    2023.03.15  59800  10482149    1.36      1
10   2023.03.16  59900  10611939    0.17      1
11   2023.03.17  61300  14090110    2.34      1
12   2023.03.20  60200   9618009   -1.79      0
13   2023.03.21  60300   8318514    0.17      1
14   2023.03.22  61100   8978591    1.33      1
15   2023.03.23  62300  15381057    1.96      1
16   2023.03.24  63000  18278602    1.12      1
17   2023.03.27  62100  11039331   -1.43      0
18   2023.03.28  62900  11614118    1.29      1
19   2023.03.29  62700  11216008   -0.32

In [ ]:
print(df_merged.head())
df_merged['감성점수'] = df_merged['감성점수'].round(2)

df_merged.to_csv('data_preprocessing.csv', index=False)


          날짜     종가       거래량  변동률(%)  상승 여부  감성점수
0 2023-03-02  60800  13095682    0.00      0  0.55
1 2023-03-03  60500  10711405   -0.49      0  0.27
2 2023-03-06  61500  13630602    1.65      1  0.50
3 2023-03-07  60700  11473280   -1.30      0  0.16
4 2023-03-08  60300  14161857   -0.66      0 -0.01
